In [1]:
import pandas as pd

In [2]:
dataset=pd.read_parquet(r'C:\Users\User\OneDrive\Documents\Trabajo\Henry\Proyecto Ind. 1\Datos Limpios\full_data.parquet')
df_developer=pd.read_parquet(r'C:\Users\User\OneDrive\Documents\Trabajo\Henry\Proyecto Ind. 1\Datos Limpios\steam_games.parquet')

In [10]:
df_1=df_developer[['developer','release_date','price','title']]
df_1=df_1.drop_duplicates(subset=['developer','title']).copy()

In [5]:
def developer_info(developer: str):
    df=df_1[df_1['developer']==developer]
    if developer not in list(df_1['developer']):
        return {'El desarrollador no se encuentra en la base de datos'}
    else:
        fechas=df['release_date'].unique()
        anio={} 
        free={} 

        for x in fechas:
            filter_condition = (df['release_date'] == x) & (df['developer'] == developer) 
            developer_releases = df[filter_condition]
            if len(developer_releases) != 0:
                anio[x] = len(developer_releases)
                free[x] = len(developer_releases[developer_releases['price'] == 0.0])
        
        for y, i in free.items():
            free[y]=f'{round((i/anio[y])*100,2)}%'
        
        resultado = pd.DataFrame({'Año': anio.keys(),'Cantidad_items': anio.values(),'Contenido_Free': free.values()})
    return resultado.sort_values(by='Año',ascending=False)

In [6]:
developer_info('Valve')

,Año,Cantidad_items,Contenido_Free
5,2017,2,0.0%
4,2016,1,100.0%
3,2012,1,0.0%
2,2011,1,0.0%
1,2010,2,50.0%
6,2009,1,0.0%
7,2008,1,0.0%
8,2007,3,33.33%
10,2006,2,0.0%
9,2004,5,0.0%


In [7]:
dataset.head()

,genres,title,release_date,price,item_id,developer,user_id,recommend,sentiment_analysis,items_count,playtime_forever,playtime_2weeks
6584043,Action,BioShock Infinite,2013,29.99,8870,"Irrational Games,Aspyr (Mac),Virtual Programmi...",Gaaaaary,1,2,213,48.0,0.0
844270,RPG,Vindictus,2012,0.00,212160,Nexon,darkedge119,1,2,153,247.0,0.0
2942786,Indie,Starbound,2016,14.99,211820,Chucklefish,razarealm,1,2,156,416.0,0.0
5049461,Action,Murdered: Soul Suspect,2014,29.99,233290,Airtight Games,mysterd,1,1,1588,1801.0,0.0
8303026,Action,Counter-Strike: Source,2004,19.99,240,Valve,Pyromanic,1,2,41,234.0,1.0


In [11]:
df_2=dataset[['user_id','title','price','items_count','recommend']]
df_2=df_2.drop_duplicates(subset=['user_id','title'], keep='first').copy()

In [9]:
def user_info(user: str):
    try:
        if user not in df_2['user_id'].unique():
            return {f'El usuario {user}, no existe.'}
        else:
            df=df_2[df_2['user_id']==user]
            resultado = {
                'Usuario': user,
                'Dinero gastado': df['price'].sum(),
                'cantidad de items': df['items_count'].head(1).to_list()[0],
                'Porcentaje de recomendaciones': f'{float(round((df['recommend'].sum()/df["items_count"].head(1).to_list()[0])*100,2))}%'
            }

        return resultado
    except:
        return {'El usuario no se encuentra en la base de datos.'}

In [10]:
usuarios_2=df_2['user_id'].unique()
diccionario={}
for i in usuarios_2[:5]:
    diccionario[i]=user_info(i)
diccionario

{'Gaaaaary': {'Usuario': 'Gaaaaary',
  'Dinero gastado': 174.95000000000002,
  'cantidad de items': 213,
  'Porcentaje de recomendaciones': '1.88%'},
 'darkedge119': {'Usuario': 'darkedge119',
  'Dinero gastado': 0.0,
  'cantidad de items': 153,
  'Porcentaje de recomendaciones': '0.65%'},
 'razarealm': {'Usuario': 'razarealm',
  'Dinero gastado': 154.93,
  'cantidad de items': 156,
  'Porcentaje de recomendaciones': '4.49%'},
 'mysterd': {'Usuario': 'mysterd',
  'Dinero gastado': 114.93999999999998,
  'cantidad de items': 1588,
  'Porcentaje de recomendaciones': '0.38%'},
 'Pyromanic': {'Usuario': 'Pyromanic',
  'Dinero gastado': 19.99,
  'cantidad de items': 41,
  'Porcentaje de recomendaciones': '2.44%'}}

In [12]:
df_3=dataset[['developer','release_date','title','user_id','recommend','sentiment_analysis']]
df_3=df_3.drop_duplicates(subset=['developer','title','user_id'], keep='first').copy()

In [13]:
df_3['release_date']=df_3['release_date'].astype(int).copy()

In [14]:
def developer_of_year(year: int):
    if year in df_3['release_date'].unique():
        anio=df_3[df_3['release_date']==year].sort_values(by='release_date', ascending=False)
        anio['rating']=anio['recommend']+anio['sentiment_analysis']
        respuesta=anio.groupby(by='developer')['rating'].sum().reset_index()
        respuesta.columns=['Desarrollador','Cantidad']
        respuesta=respuesta.sort_values(by='Cantidad', ascending=False).head(3).reset_index(drop=True)
        respuesta.index+=1
        return respuesta
    else:
        respuesta = f'Error: inserte un año que se encuentre en el rango {df_3['release_date'].min()} - {df_3['release_date'].max()}'
    return respuesta

In [15]:
developer_of_year(2014)

,Desarrollador,Cantidad
1,Edge of Reality,695
2,Coffee Stain Studios,501
3,New World Interactive,423


In [16]:
df_4=dataset[['developer','user_id','sentiment_analysis']]
df_4=df_4.drop_duplicates(subset=['developer','user_id'],keep='first').copy()

In [16]:
def review_developer(dev: str):
    try:
        df=df_4[df_4['developer']==dev]
        positivo=df[df['sentiment_analysis']==2].shape[0]
        negativo=df[df['sentiment_analysis']==0].shape[0]
        respuesta = pd.DataFrame([{'Desarrollador':dev,'Positivo':positivo,'Negativo':negativo}])
        return respuesta
    except:
        return {'No ingreso un valor relevante, o el desarrollador no tiene Resenias'}

In [17]:
review_developer('Valve')

,Desarrollador,Positivo,Negativo
0,Valve,2149,511


In [18]:
df_5=pd.read_parquet(r'C:\Users\User\OneDrive\Documents\Trabajo\Henry\Proyecto Ind. 1\Datos varios\criterio_3.parquet')
df_5.head()

,user_id,playtime_forever,genres,release_date
704300,kanzabrand,1729.0,Massively Multiplayer,2011
618527,76561198077684524,63.0,Action,2011
734375,76561198068727361,2400.0,Simulation,2012
6420286,PotatoMan1,2876.0,Casual,2013
2896219,TraiHard,10.0,Adventure,2016


In [19]:
def user_for_genre(gen: str):
    try:
        df=df_5[df_5['genres']==gen]
        usuario= df.groupby('user_id')['playtime_forever'].sum().idxmax(0)
        df_full=df[df['user_id']==usuario]
        horas=df_full.groupby('release_date')['playtime_forever'].sum().reset_index()
        horas.sort_values(by='release_date',ascending=False,inplace=True)
        horas.columns=['Año','Tiempo']
        horas_str = horas.to_string(index=False)
        return f'El usuario con mayor tiempo acumulado jugando juegos del genero {gen} es: {usuario}\n {horas_str}'
    except:
        return 'Error: el valor ingresado no se encuentra en el dataset o valor incorrecto'

In [20]:
mensaje=user_for_genre('Action')
print(mensaje)

El usuario con mayor tiempo acumulado jugando juegos del genero Action es: REBAS_AS_F-T
  Año   Tiempo
2012 350369.0
2011 326821.0
